In [14]:
import torch
from functorch.experimental.control_flow import cond, map
def test_export_cond_preserve_stack_trace_for_subgraphs():
    class MySubModule(torch.nn.Module):
        def foo(self, x):
            return x.cos()

        def forward(self, x):
            return self.foo(x)

    class CondBranchClassMethod(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.subm = MySubModule()

        def bar(self, x):
            return x.sin()

        def forward(self, x):
            return cond(x.sum() <= 2, self.subm.forward, self.bar, [x])


    from torch._export import capture_pre_autograd_graph

    example_inputs = (torch.randn(1, 3, 3, 3),)
    m = CondBranchClassMethod()
    m.eval()
    gm = capture_pre_autograd_graph(m, example_inputs)
    gm.print_readable()
    return gm

In [15]:
gm = test_export_cond_preserve_stack_trace_for_subgraphs()

class GraphModule(torch.nn.Module):
    def forward(self, x):
        arg0: f32[1, 3, 3, 3], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
        # File: /tmp/ipykernel_97542/2299104986.py:20, code: return cond(x.sum() <= 2, self.subm.forward, self.bar, [x])
        sum_default: f32[] = torch.ops.aten.sum.default(arg0)
        le_scalar: b8[] = torch.ops.aten.le.Scalar(sum_default, 2);  sum_default = None
        true_graph_0 = self.true_graph_0
        false_graph_0 = self.false_graph_0
        cond: f32[1, 3, 3, 3] = torch.ops.higher_order.cond(le_scalar, true_graph_0, false_graph_0, [arg0]);  le_scalar = true_graph_0 = false_graph_0 = arg0 = None
        return pytree.tree_unflatten([cond], self._out_spec)
        
    class <lambda>(torch.nn.Module):
        def forward(self, arg0_1: f32[1, 3, 3, 3]):
            # File: /tmp/ipykernel_97542/2299104986.py:20, code: return cond(x.sum() <= 2, self.subm.forward, self.bar, [x])
            cos: f32[1, 3, 3, 3] = torch.ops.at

In [16]:
g = torch.fx.passes.graph_drawer.FxGraphDrawer(gm, "condition_model")
g.get_dot_graph().write_svg("/home/users/yushu.gao/shared/condition_model.svg")